In [7]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','DiabetesPedigreeFunction', 'Age']
noOfIterations = 100

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else 
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputDiabetesKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsDiabetesKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputDiabetesTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsDiabetesTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputDiabetesLIMELOOCV_100.xlsx")
    sheet = dataframe.active
    feature_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','DiabetesPedigreeFunction', 'Age']
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsDiabetesLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
        max = similarityMatrix(MSP, noOfRows)
        min = similarityMatrix(MS, noOfRows)
    else:
        max = similarityMatrix(MS, noOfRows)
        min = similarityMatrix(MSP, noOfRows)
    return min / max
    #return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    # MSP = pd.DataFrame(columns = feature_names)
    # for i in range(0, noOfRows):
    #     MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MP_copy, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsDiabetesKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsDiabetesTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsDiabetesLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsDiabetesMPMS_MinMax.xlsx")

[1.0, 0.8529322363101685, 0.8529322363101685, 1.0]
[1.0, 0.7992561360194493, 0.7992561360194493, 1.0]
[1.0, 0.5139157707824377, 0.5139157707824377, 1.0]
[1.0, 0.7303715093942544, 0.7303715093942544, 1.0]
[1.0, 0.8151696229458787, 0.8151696229458787, 1.0]
[1.0, 0.859741020226912, 0.859741020226912, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:120: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:145: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[nan, nan, nan, 1.0]
[1.0, 0.8821908032505447, 0.8821908032505447, 1.0]
[1.0, 0.6833163831348599, 0.6833163831348599, 1.0]
[1.0, 0.9373439982544827, 0.9373439982544827, 1.0]
[1.0, 0.7691330189811443, 0.7691330189811443, 1.0]
[1.0, 0.6682475996427619, 0.6682475996427619, 1.0]
[1.0, 0.5295338602937484, 0.5295338602937484, 1.0]
[1.0, 0.8322308243332783, 0.8322308243332783, 1.0]
[1.0, 0.7846243140410141, 0.7846243140410141, 1.0]
[1.0, 0.9823002285448057, 0.9823002285448057, 1.0]
[1.0, 0.6905251113890154, 0.6905251113890154, 1.0]
[1.0, 0.8253634031037865, 0.8253634031037865, 1.0]
[1.0, 0.636337511250203, 0.636337511250203, 1.0]
[1.0, 0.7185489646662588, 0.7185489646662588, 1.0]
[1.0, 0.5001958327904507, 0.5001958327904507, 1.0]
[1.0, 0.6223407302510591, 0.6223407302510591, 1.0]
[1.0, 0.6135667102086387, 0.6135667102086387, 1.0]
[1.0, 0.5377268313634843, 0.5377268313634843, 1.0]
[1.0, 0.49439430503701826, 0.49439430503701826, 1.0]
[1.0, 0.8304667080967161, 0.8304667080967161, 1.0]
[1.0, 0.49

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:120: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:136: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:145: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.4983833804400871, 0.4983833804400871, 1.0]
[1.0, 0.5280399869335766, 0.5280399869335766, 1.0]
[1.0, 0.7496914870492095, 0.7496914870492095, 1.0]
[1.0, 0.8290036776582684, 0.8290036776582684, 1.0]


...

[1.0, nan, nan, nan]
[1.0, 0.7729394755545761, 0.7729394755545761, 1.0]
[1.0, 0.7912956871430215, 0.7912956871430215, 1.0]
[1.0, 0.5227769640167923, 0.5227769640167923, 1.0]
[1.0, 0.512118908768818, 0.512118908768818, 1.0]
[1.0, 0.9394220664329528, 0.9394220664329528, 1.0]
[1.0, 0.7937517459452853, 0.7937517459452853, 1.0]
[1.0, 0.4982084189809011, 0.4982084189809011, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:120: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:136: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:145: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.6193047569341418, 0.6193047569341418, 1.0]
[1.0, 0.6207957287777636, 0.6207957287777636, 1.0]
[1.0, 0.6591020045155295, 0.6591020045155295, 1.0]
[nan, nan, nan, 1.0]
[1.0, 0.6972017102981154, 0.6972017102981154, 1.0]
[nan, nan, nan, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:120: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:136: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_18644\3747841283.py:145: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.983787880249554, 0.983787880249554, 1.0]
[1.0, 0.50516392581184, 0.50516392581184, 1.0]
[1.0, 0.7278060277373041, 0.7278060277373041, 1.0]
[1.0, 0.4993800742930212, 0.4993800742930212, 1.0]
[1.0, 0.771523320685746, 0.771523320685746, 1.0]
[1.0, 0.863636497482754, 0.863636497482754, 1.0]
[1.0, 0.7259968078035988, 0.7259968078035988, 1.0]
[1.0, 0.6711802897951349, 0.6711802897951349, 1.0]
[1.0, 0.4986327198476952, 0.4986327198476952, 1.0]
[1.0, 0.6679405878308566, 0.6679405878308566, 1.0]
[1.0, 0.6231775888391119, 0.6231775888391119, 1.0]
[1.0, 0.6725346924890695, 0.6725346924890695, 1.0]
[1.0, 0.7646713163507483, 0.7646713163507483, 1.0]
[1.0, 0.8087109926041939, 0.8087109926041939, 1.0]
[1.0, 0.8527758776275682, 0.8527758776275682, 1.0]
[1.0, 0.6754284134474285, 0.6754284134474285, 1.0]
